# Creación del dataset:
Recopilación de datos:
    Inicialmente se utiliza la API de Spotify para recopilar datos sobre las características de las canciones.

Pero primero, se importan las librerías que se utilizaran en todo el proyecto.
Además, como se va a trabajar con la api de spotify se deben cargar las credenciales para posteriormente realizar las requests correspondientes. Las credenciales las paso por el archivo "config.ini" para que no queden definidas en el notebook.

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from numpy.random import shuffle 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sentiment_analysis_spanish import sentiment_analysis # Supervised Machine Learning Based Aproach
import re
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

# Spotify API credentials
client_id = config.get('spotify','client_id')
client_secret = config.get('spotify','client_secret')

# Spotify API init
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

 Ahora, defino algunas funciones para hacer estas request de manera más automatizada. Cada una tiene sus respectivos comentarios de como funciona pero a groso modo el método "get_artist_info" retorna un diccionario con el ID del artista, de sus álbumes, géneros y el id de sus canciones. Con las ids de las canciones vamos a poder hacer, mediante "get_track_features", las request de la información de cada canción para la elaboración de este dataframe.

In [2]:
def get_artist_info(artist_name):
    """
    This function takes an artist name as input and returns a dictionary containing the artist's ID and genre (if available).

    Args:
        artist_name (str): The name of the artist to search for.

    Returns:
        dict: A dictionary containing the artist's ID and genre (or None if not found).
    """
    try:
        search_results = sp.search(q=artist_name, type='artist')
        artists = search_results['artists']['items']

        if artists:
            artist_info = artists[0]  
            artist_id = artist_info['id']
            genres = artist_info.get('genres', [])  
            albums_results = sp.artist_albums(artist_id)
            album_ids = [album['id'] for album in albums_results['items']]

            album_tracks = {}
            total_tracks = []  

            # Loop through each album ID
            for album_id in album_ids:
                # Get album tracks (limit to 50 tracks per request)
                tracks_results = sp.album_tracks(album_id, limit=50)
                tracks = tracks_results['items']

                while tracks_results['next']:
                    tracks_results = sp.next(tracks_results)
                    tracks.extend(tracks_results['items'])
                    
                album_tracks[album_id] = [track['id'] for track in tracks]
                total_tracks.extend(album_tracks[album_id])

            return {
                'artist_id': artist_id,
                'genres': genres,
                'album_ids': album_ids,
                'album_tracks': album_tracks,
                'total_tracks': total_tracks
            }
    
    except:
        print(f"Error al hacer la solicitud de info de artista en la API de Spotify")
        return None

print(get_artist_info("babasonicos")) # A modo de ejemplo imprimo el return de Babass

{'artist_id': '2F9pvj94b52wGKs0OqiNi2', 'genres': ['argentine rock', 'electronica argentina', 'latin alternative', 'latin rock', 'pop electronico', 'rock en espanol'], 'album_ids': ['7g8RJEDSBDwG7Xz3s0ncq1', '5oLaRXGevqs8RYxXhYe9PG', '2vHwdJI0FKurPPmHjXj24Z', '5c6b4NSsJHroOvLFUNBE3t', '7AhXiUL10VFtwh8NY0sRQV', '37T4qeUVTpx7Ps3HhSKdKc', '6eWrCdzfRUvNMDYWGHIExi', '1x93LZe2PKKrgePH2GZjCP', '674AkakAvjA3EnJayTQ4Qb', '0h9swlJeoJLVHmuxBoCi62', '1u9PRTF6cfvCzZBYcXv6uV', '0DUCdIMPrlfPFPPKsNyzx3', '1IXP3t3bPcgiSNSVKWu6Du', '4oRMvqERfDZZGf56WF5v2q', '15eHesyZJpvolTrSUJgoDX', '7FYLw9fTOiYnJFbFk2Mntn', '1EKYYcWJkAYQ78GNYhYrJv', '7EQlXvy03jQfNjy3Ff8jGb', '2VMsGuuC4CDcyZ8qp7njWh', '5ZfZ0PWqGTRHmOxkIkQaLE'], 'album_tracks': {'7g8RJEDSBDwG7Xz3s0ncq1': ['7kb9NS6gItiOUc0ScsbnBP', '60Gx4M8cwqCV2TtAKop2iY', '5rp2y5UbMOvc7S3MCKrjI4', '5MULRnOqlPreeiOufzx7Kf', '0p5aYD8fu2jvDtawm2inOA', '3Rk611iAyxHtCWLVVgxs6b', '6uYefVt4V9eSEyuDFv2Cu9', '518GqXyTNyUxpNRPorJba6', '0UqraDB2ofMR56fDAbZRVJ', '6txmX4Wnc1FLnH13Ls

In [62]:
def get_track_features(track_id):
    """
    This function takes track id as input and returns a dictionary with track info.

    Args:
        track_id (str): The ID of the track search for.

    Returns:
        dict: A dictionary containing the features and attributes of the track.
    """
    try:
        track_info = sp.track(track_id)
        features = sp.audio_features(track_id)[0]
    
        return {
            'track_id': track_info['id'],
            'track_name': track_info['name'],
            'artist_name': track_info['artists'][0]['name'],
            'popularity': track_info['popularity'],
            'danceability': features['danceability'],
            'energy': features['energy'],
            'key': features['key'],
            'loudness': features['loudness'],
            'mode': features['mode'],
            'speechiness': features['speechiness'],
            'acousticness': features['acousticness'],
            'instrumentalness': features['instrumentalness'],
            'liveness': features['liveness'],
            'valence': features['valence'],
            'tempo': features['tempo'],
            'duration_seg' : features['duration_ms']/1000
        }
    except:
        print(f"Error when requesting track information in Spotify API")
        return None


In [ ]:

def create_dataframe(artist_names):
    """
    This function takes a list of artist names and returns a DataFrame containing combined track features for all artists.

    Args:
        artist_names (list): A list containing names of artists to search for.

    Returns:
        DataFrame: A DataFrame containing combined track features.
    """

    # Initialize an empty DataFrame to store combined results
    df = pd.DataFrame()

    # Loop through each artist name
    for artist_name in artist_names:
        artist_data = get_artist_info(artist_name)  # Get artist information

        # Extract track IDs and features
        tracks_data = [get_track_features(track_id) for track_id in artist_data['total_tracks']]

        # Create a temporary DataFrame for this artist's tracks
        artist_df = pd.DataFrame(tracks_data)

        # # Add a 'genre' column and fill with the first genre (assuming single genre)
        # artist_df['genre'] = 0
        # artist_df['genre'].fillna(artist_data['genres'][0], inplace=True)  # Assuming first artist's genre

        # Concatenate the artist's DataFrame to the combined DataFrame
        df = pd.concat([df, artist_df], ignore_index=True)  # Avoid duplicate indices

        #To delete the infiltrators artist
        # unwanted_artist = df[~df["artist_name"].isin(artist_names)].index
        # df.drop(unwanted_artist, inplace=True)

    return df

# Call the function after retrieving artist 
#"Trueno","Airbag","Duki","Callejeros","Acru","Ciro y los Persas","YSY A","Frescolate"
#"La Mona Jimenez","Damas Gratis","Nestor en Bloque","Nene malo","Los Palmeras","Ulises Bueno","Luck Ra","la K'onga",""
artist_names = ["Los Pibes Chorros"]
df = create_dataframe(artist_names)


In [ ]:
df['artist_name'].value_counts()

In [ ]:
unwanted_artist = df[~df["artist_name"].isin(artist_names)].index
df.drop(unwanted_artist, inplace=True)

df['artist_name'].value_counts()

Como los artistas están siendo elegidos por mi, teniendo en cuenta lo que considero artistas representantes específicamente de su género, designó el tag de género de esta forma. También se puede hacer de manera automática con el género de cada artista que nos devuelve la API, pero en múltiples casos este valor no estaba bien designado o era nulo, por eso decidí hacerlo de manera manual. 

In [ ]:
# Genre. 0 = Cuarteto ; 1 = Cumbia
df['genre'] = 1
# indice =  df['artist_name'] == artist_names  
# df.loc[indice, 'genre'] = 1

df['genre'].value_counts()

Ahora solo falta juntar esta nueva informacion con la info de otros artistas ya analisados, para ello guardo los valores en 'data.csv'

In [ ]:
df_old = pd.read_csv('data.csv')
# Add the new artist into the original DataFrame
df_new = pd.concat([df_old, df], ignore_index=True)

# Save the DataFrame into 'data.csv'
df_new.to_csv('data.csv', index=False)


# Nueva Feature

En este punto ya se creó el dataset, se guardó artista por artista en el .csv. Pero resultaría interesante agregar una entrada más, que hable un poco de la letra de estas canciones. Ya que al fin y al cabo las features obtenidas de spotify habla en general de la música pero no del contenido de la letra en sí mismo.

Para esto se realizará un análisis sentimental de cada letra, lo primero es leer el dataset y quiero ver que los registros estén balanceados.


In [6]:
df_spotify = pd.read_csv('data.csv')
df_spotify['genre'].value_counts()


genre
0    1041
1     904
Name: count, dtype: int64

Lo primero que voy a necesitar es la letra de cada canción, spotify no me provee de esta info asi que ya no voy a poder recurrir a la API. 

No queda otra que aventurarse en las oscuras artes del scraping, la página letras.com parece ser ideal para esto.

Utilizando BeautifulSoup e inspeccionando un poco el html de la página logró crear la función "obtener_letra" que me devuelve la letra de cada una de las canciones.

In [63]:
import requests
from bs4 import BeautifulSoup

def obtener_letra(artista, titulo):
  """
  Función para obtener la letra de una canción en Letras.com.

  Args:
    artista: Nombre del artista.
    titulo: Título de la canción.

  Returns:
    La letra de la canción como string, o None si no se encuentra.
  """
  artista = artista.lower().replace(" ","-")
  # titulo = re.sub(r"\s*\(.*?\)\s*", "", titulo)
  titulo = re.sub(r"[\(\[].*?[\)\]]", "", titulo)
  titulo = re.sub(r"/.*", "", titulo)
  titulo = titulo.lower().replace(" ","-")


  url = f"https://www.letras.com/{artista}/{titulo}"
  response = requests.get(url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    letra_div = soup.find('div', class_ = "lyric-original font --lyrics --size18")

    if letra_div:
      return letra_div.text.strip()
    else:
      return None
  else:
    print(f"Error al acceder a la URL: {url}")
    return None

# Ejemplo de uso
artista = "damas-gratis"
titulo = "sos-boton"

letra = obtener_letra(artista, titulo)

if letra:
  print(f"Letra de {titulo} de {artista}:\n{letra}")
else:
  print(f"No se encontró la letra de {titulo} de {artista}")

Letra de sos-boton de damas-gratis:
¡Sufre, por hacerte el policia!¡No!, no lo puedo creerVos ya no sos el vagoYa no sos el atorranteAl que los pibes lo llamaban el picanteAhora te llaman botonYa no estásCon tus amigosY en la esquina te la dabasDe polenta, de malevo y de matonY solo eras un botonY solo eras un botonVos sos un botonNunca vi a un policiaTan amargo como vosVos sos un botonNunca vi a un policiaTan amargo como vos¡Damas gratis, wipi!Para vos, CarlitosCuando ibas a la canchaParabas con la hinchadaY tomabas vino blancoY ahora patrullas la ciudadSi vas a la cancha vas con celularY a tus amigosAndas arrestandoSos el policia del comandoVos sos un botonNunca vi a un policiaTan amargo como vosVos sos un botonNunca vi a un policiaTan amargo como vosVos sos un botonNunca vi a un policiaTan amargo como vosVos sos un botonNunca vi a un policiaTan amargo como vos


La letra que nos provee letras.com tiene algunos problemas de formato, para esto creo el metodo "agregar_espacios" asi la acomodamos un poco

In [ ]:
def agregar_espacios(texto):
  """
  Función para agregar espacios antes de mayúsculas en un texto usando expresiones regulares.

  Args:
    texto: El texto original.

  Returns:
    El texto con espacios agregados antes de mayúsculas.
  """
  if texto:
    return re.sub(r"([a-z])([A-Z])", r"\1. \2", texto)
  else:
    return None

El metodo "sentimental_coeficient" implementa el objeto sentiment_analysis y obtiene el valor de "positividad" de cada una de las frases de la cancion para luego promediarlas y lograr un valor general de positividad del track.

In [ ]:
def sentimental_coeficient(letra):
    """
    Funcion que retorna el valor de sentimiento obtenido del promedio en las frases de la 
    Args: 
        letra: Letra de la cansion

    Returns:
        La media de los sentimientos donde 1 es alegría y 0 tristeza
    """
    if letra:
        letra = agregar_espacios(letra)
        sentimental = sentiment_analysis.SentimentAnalysisSpanish() # Inicializamos el objeto
        sentimental_coef = np.array([sentimental.sentiment(f) for f in letra.split('.')])
        sentimental_mean = np.mean(sentimental_coef) 
        if sentimental_mean:
            return sentimental_mean
        else:
            return 0
    else :
        return 0

Solo falta recorrer todas las canciones en el dataset y añadir a cada registro la "positividad"

In [ ]:
for index, row in df_spotify.iterrows():
    if df_spotify["positividad"][index] == 0:
        print(row["track_name"])
        track_name = re.split(r" - ",row["track_name"])[0]
        print(track_name)
        if "!" in track_name:
            df_spotify.drop(index, inplace = True)
        else:    
            letra = obtener_letra(row["artist_name"],track_name)
            letra = agregar_espacios(letra)
            print(index)
            df_spotify.loc[index,"positividad"] = sentimental_coeficient(letra)
            print(df_spotify["positividad"][index])